In [1]:
! pip install langchain langchain_core sentence_transformers streamlit faiss-cpu langchain-community load_dotenv openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
import os

from langchain.chains import ConversationalRetrievalChain
from langchain import PromptTemplate, LLMChain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains.question_answering import load_qa_chain

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models.openai import ChatOpenAI

from sentence_transformers import CrossEncoder
from dotenv import load_dotenv

from app import ReRankingCompressor
from prompt_default import QA_PROMPT, CONDENSE_QUESTION_PROMPT


embeddings = HuggingFaceEmbeddings(
    model_name="multi-qa-mpnet-base-dot-v1"
)
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

load_dotenv()

output_parser = StrOutputParser()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
vectorstore = FAISS.load_local("vectorstore", embeddings=embeddings, allow_dangerous_deserialization=True )
retriever = vectorstore.as_retriever(search_kwargs={"k":100})


# Inference here

In [44]:
# from langchain.chat_models import ChatOpenAI
# from langchain.retrievers.document_compressors import LLMChainFilter

# os.environ["OPENAI_API_KEY"] = 'sk-'

# generation_config = {"temperature": 0.1, "top_p": 0.9}#, "max_tokens": 512}

# llm_reranking = ChatOpenAI(model_name="gpt-4o", **generation_config)
# llm_filter = LLMChainFilter.from_llm(llm_reranking)

                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


In [47]:
def qa_prompt():
  prompt_template = """<|im_start|>system
  Task Description:
    1. Read the given question and context to gather relevant information.
    2. Write reading notes summarizing the key points from the context.
    3. Discuss the relevance of the given question and context.
    4. If some context are relevant to the given question, provide a brief answer based on the context.
    5. If no passage is relevant, directly provide answer without considering the context.
  "\n\n"
  {context}
  <|im_end|>
  <|im_start|>user
  {question}
  <|im_end|>
  <|im_start|>assistant"""

  prompt = PromptTemplate(
  template=prompt_template, input_variables=["context", "question"]
  )
  return prompt


In [115]:
model_kwargs = {
    "max_tokens": 512,
    "stop": ['://', 'https://'],
    "frequency_penalty": 0.1, 
}

llm = ChatOpenAI(
    openai_api_key="hf_",
    openai_api_base="https://jjs5s9795wiabm9z.us-east-1.aws.endpoints.huggingface.cloud" + "/v1/",
    **model_kwargs
)

                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.
                    frequency_penalty was transferred to model_kwargs.
                    Please confirm that frequency_penalty is what you intended.


In [117]:
import pandas as pd
import time

df=pd.read_csv('QA.csv')
df = df[['url','question','answer']]

for index, row in df.iterrows():

    query = row['question'] 
    print('\nQUERY: ',row['question'])

    question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT())
    doc_chain = load_qa_chain(
                        llm, chain_type="stuff", prompt=qa_prompt()
                    )
    re_ranking_compressor = ReRankingCompressor(
        cross_encoder=cross_encoder, topk=5, llm_chain_ranking=None #llm_filter
    )
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=re_ranking_compressor, base_retriever=retriever
    )
    qa_chain = ConversationalRetrievalChain(
        retriever=compression_retriever, combine_docs_chain=doc_chain, question_generator=question_generator
    )

    result = qa_chain.run({
                            "question": query,
                            "chat_history": [],
                        },
                    )
    df.at[index, 'prediction'] = result
    print('\nANSWER: ',result)

df.to_csv('TEST_mymodel.csv', index=False)



QUERY:  What is the total amplitude expected for the market adjustment according to ABS?

ANSWER:     ### Reading Notes:

1. **Latest ABS Report**:
   - The market is expected to undergo an adjustment starting from early April.
   - The amplitude of the adjustment is 120 points to 1,166 points.
   - Most stock groups will experience an average discount of approximately 15%.

2. **Second Rhythm Adjustment**:
   - The market is likely to continue adjusting in the second rhythm.
   - The total amplitude of the medium-term adjustment is expected to be 160 points from the peak area.
   - The adjustment period is estimated at 8-9 weeks.

3. **Important Trend Areas**:
   - Investors should pay attention to the areas of 1,130-1,080 points as important trend indicators.
   - The market is expected to increase slightly in the short term, with a target range of 85.75-85.

   ### Relevance of the Question and Context:

The question asks for the total amplitude expected for the market adjustment a